## How to Apply LLMs to Extract Variables from Unstructured Text for Real-World Data Analysis?

The goal of this project is to identify patients with potential MASLD by extracting fatty liver findings from unstructured ultrasound reports. I used SQL and regex to retrieve liver-related lines from semi-structured narrative data stored in Oracle (Epic format), and applied the ChatGPT API to classify whether each report indicates steatosis. This approach demonstrates how large language models can be used to structure free-text clinical data for downstream risk identification and care management.

## Example

**Project: Extracting Fatty Liver Indicators from Ultrasound Reports Using ChatGPT API**

This project demonstrates how I used the **ChatGPT API** to extract fatty liver indicators from **unstructured ultrasound reports** to help identify MASLD patients for care management.

Background: MASLD affects ~25% of adults and can lead to chronic conditions like MASH, cirrhosis, and cardiovascular disease.

Challenge: Fatty liver is often missing from structured data. While ultrasound detects it, findings are buried in **semi-structured narrative reports stored in Oracle SQL tables**, following Epic’s standard format. I used **SQL and regex** to isolate liver-related lines and applied **GPT** to classify whether steatosis was present.


## My solution

This project extracts evidence of fatty liver from unstructured ultrasound reports using a multi-step process combining SQL, regex filtering, and the ChatGPT API.

**Step 1: Data Extraction**
Using SQL, I identified patients who received abdominal ultrasounds based on procedure codes and extracted their corresponding narrative reports, which were stored line by line.

**Step 2: Text Filtering**
I applied regular expressions (regex) within SQL to extract only the lines relevant to liver findings. Because the narrative follows a consistent semi-structured format (with lines labeled LIVER: or containing liver-related terms like echogenicity), regex provided a precise and efficient way to isolate the relevant content for further analysis — avoiding unnecessary processing of unrelated sections.

**Step 3: LLM Classification**
I used the ChatGPT API (GPT-4o-mini) with a structured prompt to determine whether the filtered text indicated fatty liver (steatosis). The model returned a simple "Yes" or "No" classification.

**Step 4: Validation**
I compared model output to physician-reviewed labels and achieved 95% accuracy, demonstrating the reliability of this lightweight, prompt-based NLP method.

NOTE:

1. The narrative data used in this project is fully simulated to mimic real-world clinical formatting and terminology. No actual patient data was used, and the dataset contains no protected health information (PHI).

2. To run the GPT classification component, you must provide your own OpenAI API key.
Set it as an environment variable by adding the following line to your ~/.bash_profile (or ~/.zshrc if using zsh):

export OPENAI_API_KEY=your_key_here


In [13]:
import pandas as pd
from openai import OpenAI

client = OpenAI()

# Load the simulated data (no actual patient data was used and no PHI in dataset)
df = pd.read_excel('simulated_ultrasound_reports.xlsx')

# Function to call OpenAI GPT API and classify fatty liver
def check_fatty_liver(text):
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a medical assistant. Answer 'Yes' or 'No' based on whether the ultrasound report mentions fatty liver (steatosis)."},
                {"role": "user", "content": f"Does this patient have fatty liver? {text}"}
            ]
        )
        result = response.choices[0].message.content.strip().lower()
        if "yes" in result:
            return "Yes"
        elif "no" in result:
            return "No"
        else:
            return "Uncertain"
    except Exception as e:
        print(f"Error processing text: {e}")
        return "Error"

# Apply the function to each narrative in the dataframe
df['predicted_fatty_liver'] = df['narrative'].apply(lambda x: check_fatty_liver(x))

# Create a new column that checks for matching rows
df['match'] = ((df['predicted_fatty_liver'].str.lower() == 'yes') & (df['fatty_liver'] == 'Y')) | \
              ((df['predicted_fatty_liver'].str.lower() == 'no') & (df['fatty_liver'] == 'N'))

# Print summary
num_matches = df['match'].sum()
total_rows = len(df)
percentage_matches = (num_matches / total_rows) * 100

print(f"Number of matches: {num_matches}")
print(f"Total number of rows: {total_rows}")
print(f"Percentage of matches: {percentage_matches:.2f}%")

# Display the final DataFrame
df



Number of matches: 10
Total number of rows: 10
Percentage of matches: 100.00%


,narrative,fatty_liver,predicted_fatty_liver,match
0,LIVER: Demonstrates increased echogenicity con...,Y,Yes,True
1,LIVER: Mild diffuse steatosis is noted.,Y,Yes,True
2,LIVER: Findings suggest moderate hepatic steat...,Y,Yes,True
3,"LIVER: Enlarged and hyperechoic, compatible wi...",Y,Yes,True
4,LIVER: Shows increased echogenicity without fo...,Y,Yes,True
5,LIVER: Diffusely echogenic appearance indicati...,Y,Yes,True
6,LIVER: Parenchymal echotexture is consistent w...,Y,Yes,True
7,LIVER: Displays hyperechoic pattern typical of...,Y,Yes,True
8,LIVER: Appears mildly enlarged with increased ...,Y,Yes,True
9,LIVER: Echogenicity elevated relative to kidne...,Y,Yes,True


### Impact - Potential applications include: 

This pipeline enables scalable extraction of fatty liver evidence from unstructured ultrasound reports, by integrating fatty liver indicators into structured data workflows has significant downstream applications:

**1. Risk Adjustment & HCC Capture**  
MASLD is a chronic, resource-consuming condition that often leads to comorbidities like diabetes, cardiovascular disease, and cirrhosis — all of which affect risk scores in value-based care models. Identifying steatosis early helps ensure that patient complexity is accurately reflected, especially for Medicare Advantage and ACO populations.


**2. Care Targeting & Pathway Optimization**  
Patients flagged with steatosis but no diagnosis code can be prioritized for lifestyle interventions, nutrition support, GLP-1 evaluation, or fibrosis assessment, improving long-term outcomes and potentially lowering downstream utilization.


**3. Real-World Evidence (RWE) Generation**  
Enhances the quality of retrospective analyses by identifying MASLD cases that would otherwise be excluded — enabling more complete studies of treatment patterns, disease burden, and healthcare utilization.


**4. Registry and Surveillance Expansion**  
Enables health systems to expand MASLD registries using unstructured reports — creating more comprehensive panels for chronic disease tracking and early-stage care coordination.


- THE END --
